In [13]:
"""
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
"""

'\n%tensorflow_version 2.x\nimport tensorflow as tf\nprint("Tensorflow version " + tf.__version__)\n\ntry:\n  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection\n  tf.config.experimental_connect_to_cluster(tpu)\n  tf.tpu.experimental.initialize_tpu_system(tpu)\n  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)\n  print(\'Running on TPU \', tpu.cluster_spec().as_dict()[\'worker\'])\nexcept ValueError:\n  raise BaseException(\'ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!\')\n'

In [14]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
name   ="H1_2008-2022_DataFrame"
#df=pd.read_csv("EURUSD.csv")
#df = pd.read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv', engine='python',index_col=[0])
    
#print("Number of rows and columns:", df.shape)
#df = df.drop('Adj Close',axis=1)
#df = df.drop('Volume',axis=1)
#df.head(10)

In [15]:
#df_y = df.shift(-1, axis = 0)
df_y =df_y.dropna()
df_y

#df_x = df.shift(1,axis=0)
df_x = df_x.dropna()
df=0
df_x,df_y


(                      Date     Open     High      Low    Close
 1      2007-12-31 01:00:00  1.47350  1.47380  1.47320  1.47360
 2      2007-12-31 02:00:00  1.47350  1.47470  1.47320  1.47360
 3      2007-12-31 03:00:00  1.47370  1.47440  1.47350  1.47440
 4      2007-12-31 04:00:00  1.47430  1.47470  1.47420  1.47440
 5      2007-12-31 05:00:00  1.47450  1.47470  1.47420  1.47420
 ...                    ...      ...      ...      ...      ...
 86744  2021-12-31 18:00:00  1.13717  1.13793  1.13588  1.13787
 86745  2021-12-31 19:00:00  1.13790  1.13861  1.13769  1.13860
 86746  2021-12-31 20:00:00  1.13860  1.13860  1.13745  1.13842
 86747  2021-12-31 21:00:00  1.13842  1.13853  1.13793  1.13822
 86748  2021-12-31 22:00:00  1.13822  1.13847  1.13752  1.13755
 
 [86748 rows x 5 columns],
                       Date     Open     High      Low    Close
 0      2007-12-31 02:00:00  1.47350  1.47470  1.47320  1.47360
 1      2007-12-31 03:00:00  1.47370  1.47440  1.47350  1.47440
 2      200

In [16]:
from sklearn.model_selection import train_test_split
train_size = ((len(df_x)*90)//100)
X_train, X_test, y_train, y_test = train_test_split(df_x.iloc[:train_size, 1:].values, df_y.iloc[:train_size, 1:].values, test_size=0.01,shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((77292, 4), (781, 4), (77292, 4), (781, 4))

In [17]:
"""
train_size = ((len(df)*90)//100)
training_set = df.iloc[:train_size, 1:2].values
test_set = df.iloc[train_size:, 1:2].values

training_set[:5],test_set[:5]
"""

'\ntrain_size = ((len(df)*90)//100)\ntraining_set = df.iloc[:train_size, 1:2].values\ntest_set = df.iloc[train_size:, 1:2].values\n\ntraining_set[:5],test_set[:5]\n'

In [18]:
"""
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

time_steps_train =1 #60

# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(time_steps_train, train_size):
    X_train.append(training_set_scaled[i-time_steps_train:i, 0])
    y_train.append(training_set_scaled[i, 0])
    print(X_train,y_train,i-time_steps_train,i)
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)
print(y_train.shape)
print(X_train[-5:])
print(y_train[-5:])
#(740, 60, 1)
"""

'\n# Feature Scaling\nsc = MinMaxScaler(feature_range = (0, 1))\ntraining_set_scaled = sc.fit_transform(training_set)\n\ntime_steps_train =1 #60\n\n# Creating a data structure with 60 time-steps and 1 output\nX_train = []\ny_train = []\nfor i in range(time_steps_train, train_size):\n    X_train.append(training_set_scaled[i-time_steps_train:i, 0])\n    y_train.append(training_set_scaled[i, 0])\n    print(X_train,y_train,i-time_steps_train,i)\nX_train, y_train = np.array(X_train), np.array(y_train)\nX_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))\nprint(X_train.shape)\nprint(y_train.shape)\nprint(X_train[-5:])\nprint(y_train[-5:])\n#(740, 60, 1)\n'

In [19]:
"""# Getting the predicted stock price of 2017
dataset_train = df.iloc[:train_size, 1:2]
dataset_test = df.iloc[train_size:, 1:2]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - time_steps_train:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
y_test = []
for i in range(time_steps_train, len(df)-train_size+time_steps_train): #519
    X_test.append(inputs[i-time_steps_train:i, 0])
    y_test.append(inputs[i, 0])
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test.shape
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))

print(X_test.shape)
print(y_test.shape)
print(X_test[-5:])
print(y_test[-5:])

#(459, 60, 1)
"""

'# Getting the predicted stock price of 2017\ndataset_train = df.iloc[:train_size, 1:2]\ndataset_test = df.iloc[train_size:, 1:2]\ndataset_total = pd.concat((dataset_train, dataset_test), axis = 0)\ninputs = dataset_total[len(dataset_total) - len(dataset_test) - time_steps_train:].values\ninputs = inputs.reshape(-1,1)\ninputs = sc.transform(inputs)\nX_test = []\ny_test = []\nfor i in range(time_steps_train, len(df)-train_size+time_steps_train): #519\n    X_test.append(inputs[i-time_steps_train:i, 0])\n    y_test.append(inputs[i, 0])\nX_test = np.array(X_test)\ny_test = np.array(y_test)\nX_test.shape\nX_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))\n#y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))\n\nprint(X_test.shape)\nprint(y_test.shape)\nprint(X_test[-5:])\nprint(y_test[-5:])\n\n#(459, 60, 1)\n'

In [20]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 4))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

# Fitting the RNN to the Training set

In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size = 4096,validation_data=(X_test,y_test))

Epoch 1/100
19/19 [==============================] - 5s 256ms/step - loss: 0.0068 - accuracy: 0.2577 - val_loss: 1.7402e-04 - val_accuracy: 0.0218
Epoch 2/100
19/19 [==============================] - 5s 252ms/step - loss: 0.0066 - accuracy: 0.2618 - val_loss: 1.1196e-04 - val_accuracy: 0.9385
Epoch 3/100
19/19 [==============================] - 5s 254ms/step - loss: 0.0065 - accuracy: 0.2639 - val_loss: 1.1220e-04 - val_accuracy: 0.1408
Epoch 4/100
19/19 [==============================] - 5s 254ms/step - loss: 0.0063 - accuracy: 0.2579 - val_loss: 9.9920e-05 - val_accuracy: 0.5506
Epoch 5/100
19/19 [==============================] - 5s 254ms/step - loss: 0.0062 - accuracy: 0.2608 - val_loss: 6.5135e-05 - val_accuracy: 0.3380
Epoch 6/100
19/19 [==============================] - 5s 253ms/step - loss: 0.0061 - accuracy: 0.2585 - val_loss: 5.1785e-05 - val_accuracy: 0.8348
Epoch 7/100
19/19 [==============================] - 5s 255ms/step - loss: 0.0060 - accuracy: 0.2613 - val_loss: 4.641

In [22]:
#model.save('/content/drive/MyDrive/EURUSD_M1_CalisanModel/')

In [23]:
predicted_stock_price = model.predict(X_test)
#predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [24]:
predicted_stock_price[-1:],y_test[-1:]


(array([[1.1632717, 1.1574049, 1.209153 , 0.9983849]], dtype=float32),
 array([[1.49101, 1.49146, 1.49042, 1.49119]]))

In [ ]:
# Visualising the results
#plt.plot(df.loc[train_size:, "Date"],y_test, color = "red", label = "Real Price")
#plt.plot(df.loc[train_size:, "Date"],predicted_stock_price, color = "blue", label = "Predicted Price")
#plt.xticks(np.arange(0,len(df),50))
#plt.title('Price Prediction')
#plt.xlabel('Time')
#plt.ylabel('Price')
#plt.legend()
#plt.show()

In [ ]:
arr = np.empty(shape=(1,1,1,1))
arr = [1.09785,1.09785,1.09735,1.09735]
arr = np.reshape(arr,(1,4))
a = model.predict(arr)
a

#1.0982989, 1.0983827, 1.0982116, 1.0982965
#1.0982978, 1.0983816, 1.0982106, 1.0982955